In [1]:
import importlib
import sys
import getpass
import uuid
from jobmon.client.tool import Tool  # type: ignore
from pathlib import Path

import idd_climate_models.constants as rfc
from idd_climate_models.validate_model_functions import *

repo_name = rfc.repo_name
package_name = rfc.package_name
DATA_DIR = rfc.DATA_PATH
PROCESSED_DATA_PATH = rfc.PROCESSED_DATA_PATH

SCRIPT_ROOT = rfc.REPO_ROOT / repo_name / "src" / package_name / "clean_and_split"

# Toggle for rerunning already processed files
RERUN = False  # Set to True to reprocess everything, False to skip already processed files

# Validate all models
results = validate_all_models(DATA_DIR, data_source='cmip6', verbose=True)

complete_models = get_complete_models(results)
print(f"\nFully downloaded models: {len(complete_models)}")
print(complete_models)

Validating 26 climate models...
[1/26] Validating ACCESS-CM2... ✓ COMPLETE
[2/26] Validating ACCESS-ESM1-5... ✓ COMPLETE
[3/26] Validating AWI-CM-1-1-MR... ✓ COMPLETE
[4/26] Validating BCC-CSM2-MR... ✓ COMPLETE
[5/26] Validating CESM2... ✗ INCOMPLETE
[6/26] Validating CESM2-WACCM... ✓ COMPLETE
[7/26] Validating CMCC-CM2-SR5... ✓ COMPLETE
[8/26] Validating CMCC-ESM2... ✓ COMPLETE
[9/26] Validating CanESM5... ✓ COMPLETE
[10/26] Validating EC-Earth3... ✓ COMPLETE
[11/26] Validating EC-Earth3-Veg... ✓ COMPLETE
[12/26] Validating EC-Earth3-Veg-LR... ✓ COMPLETE
[13/26] Validating FGOALS-g3... ✗ INCOMPLETE
[14/26] Validating IITM-ESM... ✗ INCOMPLETE
[15/26] Validating INM-CM4-8... ✓ COMPLETE
[16/26] Validating INM-CM5-0... ✓ COMPLETE
[17/26] Validating IPSL-CM6A-LR... ✓ COMPLETE
[18/26] Validating KACE-1-0-G... ✗ INCOMPLETE
[19/26] Validating MIROC6... ✓ COMPLETE
[20/26] Validating MPI-ESM1-2-HR... ✓ COMPLETE
[21/26] Validating MPI-ESM1-2-LR... ✓ COMPLETE
[22/26] Validating MRI-ESM2-0... ✓ CO

In [2]:
# First, run the main validation function
all_results = validate_all_models(DATA_DIR, data_source='cmip6', verbose=True)
complete_models = get_complete_models(all_results)
print(f"\nFully downloaded models: {len(complete_models)}")
print(complete_models)
# Get the names of the incomplete models
incomplete_models = get_incomplete_models(all_results)

print(f"Found {len(incomplete_models)} incomplete models. Displaying issues:")

# Loop through the incomplete models and print their issues
for model_name in incomplete_models:
    model_data = all_results[model_name]
    print_model_tree(model_data, model_name)

Validating 26 climate models...
[1/26] Validating ACCESS-CM2... ✓ COMPLETE
[2/26] Validating ACCESS-ESM1-5... ✓ COMPLETE
[3/26] Validating AWI-CM-1-1-MR... ✓ COMPLETE
[4/26] Validating BCC-CSM2-MR... ✓ COMPLETE
[5/26] Validating CESM2... ✗ INCOMPLETE
[6/26] Validating CESM2-WACCM... ✓ COMPLETE
[7/26] Validating CMCC-CM2-SR5... ✓ COMPLETE
[8/26] Validating CMCC-ESM2... ✓ COMPLETE
[9/26] Validating CanESM5... ✓ COMPLETE
[10/26] Validating EC-Earth3... ✓ COMPLETE
[11/26] Validating EC-Earth3-Veg... ✓ COMPLETE
[12/26] Validating EC-Earth3-Veg-LR... ✓ COMPLETE
[13/26] Validating FGOALS-g3... ✗ INCOMPLETE
[14/26] Validating IITM-ESM... ✗ INCOMPLETE
[15/26] Validating INM-CM4-8... ✓ COMPLETE
[16/26] Validating INM-CM5-0... ✓ COMPLETE
[17/26] Validating IPSL-CM6A-LR... ✓ COMPLETE
[18/26] Validating KACE-1-0-G... ✗ INCOMPLETE
[19/26] Validating MIROC6... ✓ COMPLETE
[20/26] Validating MPI-ESM1-2-HR... ✓ COMPLETE
[21/26] Validating MPI-ESM1-2-LR... ✓ COMPLETE
[22/26] Validating MRI-ESM2-0... ✓ CO

In [ ]:

# Filter out already processed tasks if RERUN is False
if not RERUN:
    print("Filtering out already processed files...")
    results = filter_already_processed(results, PROCESSED_DATA_PATH, rerun=RERUN)
    complete_models = get_complete_models(results)
    print(f"After filtering: {len(results)} models with tasks to process:")
    print(complete_models)
else:
    print("RERUN is set to True; all files will be reprocessed.")
    complete_models = get_complete_models(results)



Fully downloaded models: 10
['ACCESS-CM2', 'ACCESS-ESM1-5', 'EC-Earth3', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM', 'TaiESM1']
Filtering out already processed files...
After filtering: 1 models with tasks to process:
['MPI-ESM1-2-LR']


In [10]:
print(f"Processing: {os.path.basename(file_path)}")

try:
    # Attempt to open the dataset but turn OFF automatic time decoding
    ds = xr.open_dataset(file_path, engine='netcdf4', decode_times=False)
    
    print("\n✅ Successfully opened with decode_times=False.")
    print("This confirms the issue is with decoding the 'time' variable's data.")
    
    print("\n--- Raw Time Variable ---")
    print(ds['time'])
    
    # You can now inspect the raw numerical values
    # print("\nFirst 10 raw time values:")
    # print(ds['time'].values[:10])

except Exception as e:
    print(f"\n❌ Failed even with decode_times=False. This indicates a more severe file issue.")
    print(e)

Processing: ua_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_19900101-20091231.nc

❌ Failed even with decode_times=False. This indicates a more severe file issue.
NetCDF: HDF error


In [2]:
# Year filtering constants
MIN_YEAR = 1950
MAX_YEAR = 2100

In [3]:
file_path = '/mnt/team/rapidresponse/pub/tropical-storms/data/raw/cmip6/MPI-ESM1-2-LR/r1i1p1f1/historical/ua/gn/day/ua_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_19900101-20091231.nc'

In [4]:
print(f"Processing: {os.path.basename(file_path)}")

# 1. Open dataset (NO CHUNKING)
ds = xr.open_dataset(file_path, engine='netcdf4')

Processing: ua_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_19900101-20091231.nc


RuntimeError: Failed to decode variable 'time': NetCDF: HDF error

In [11]:
# First, run the main validation function
all_results = validate_all_models(DATA_DIR, data_source='cmip6', verbose=False)

# Get the names of the incomplete models
incomplete_models = get_incomplete_models(all_results)

print(f"Found {len(incomplete_models)} incomplete models. Displaying issues:")

# Loop through the incomplete models and print their issues
for model_name in incomplete_models:
    model_data = all_results[model_name]
    print_model_tree(model_data, model_name)

Found 16 incomplete models. Displaying issues:

Model: AWI-CM-1-1-MR
Status: ✗ INCOMPLETE
Variant: r1i1p1f1 [✗]
  historical [✗]
    ⚠ Missing required folders: ['ua', 'va']
    tos [✗]
      gn [✗]
        Omon [✗]
          ⚠ Coverage starts too late: 197101, expected by: 197001
          📁 5 files

Model: BCC-CSM2-MR
Status: ✗ INCOMPLETE
Variant: r1i1p1f1 [✗]
  historical [✗]
    ⚠ Missing required folders: ['ua', 'va']

Model: CESM2
Status: ✗ INCOMPLETE
Variant: r1i1p1f1 [✗]
  ⚠ Missing required folders: ['ssp126', 'ssp245', 'ssp585']
  historical [✗]
    tos [✗]
      ⚠ Expected 1 folder(s), found 2: ['gn', 'gr']

Model: CESM2-WACCM
Status: ✗ INCOMPLETE
Variant: r1i1p1f1 [✗]
  historical [✗]
    tos [✗]
      ⚠ Expected 1 folder(s), found 2: ['gn', 'gr']
  ssp126 [✗]
    tos [✗]
      ⚠ Expected 1 folder(s), found 2: ['gn', 'gr']
    ua [✗]
      gn [✗]
        day [✗]
          ⚠ Coverage ends too early: 20941231, expected until: 21001231
          📁 8 files
    va [✗]
      gn [